In [24]:
import openai
import pandas as pd

client = openai.Client()


In [25]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose= "assistants"
)

In [27]:
print(file.id)

file-VgWxcrqru8FsJSxD7zzDZf


In [28]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um Analista que analisa dados sobre vendas",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}}, # Corrected line
    model="gpt-4o"
)

In [29]:
# pergunta ="Qual o rating médio das vendas do supermercado"

pergunta ="Gere um gráfico de Pizza com o percentual de vendas por linha de produtos."

In [30]:
# Criação da Thread

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role ="user",
    content = pergunta
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\143534826.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\143534826.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [31]:
# Executa a Thread

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome do usuário premium "
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\3010580843.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [32]:
import time

# Assuming 'client', 'thread', and 'run' are already defined from previous steps

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id = run.id
    )

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\3457621643.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [34]:
if run.status =="completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\827710479.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_5rXGpmUZfUowoAR8gUFxnzde', assistant_id='asst_mxqgc9bDIpPoh5X2tkdvW4L9', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-G1SBvSDnSCWApUeDuTYPKs', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza mostrando o percentual de vendas por linha de produtos. Cada segmento do gráfico representa a proporção das vendas totais que cada linha de produto contribuiu. Se precisar de mais alguma análise ou informação, sinta-se à vontade para pedir!'), type='text')], created_at=1750792350, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_7bDwKxf1R24pyGNHDU3x5Daj', status=None, thread_id='thread_Os9YZbt0DuYlgHdHEeR5WTu8'), Message(id='msg_1oiRexJWcydEZKv65xP5aZVe', assistant_id='asst_mxqgc9bDIpPoh5X2tkdvW4L9', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(a

In [35]:
# Analisando os passos do modelo

run_steps = client.beta.threads.runs.steps.list(
     thread_id=thread.id,
     run_id = run.id
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\3017233887.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [38]:
import os

# Assumindo que 'client' e 'thread' estão definidos em outra parte do seu código

for step in run_steps.data[::-1]:
    print(f"\n==== Passo: {step.step_details.type} ====")

    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 20)
            if tool_call.type == "code_interpreter":
                print(f"Entrada do Interpretador de Código:\n{tool_call.code_interpreter.input}")
            elif tool_call.type == "retrieval":
                print(f"Ferramenta de Recuperação (Retrieval) chamada.")
            elif tool_call.type == "function":
                print(f"Ferramenta de Função chamada: {tool_call.function.name} com argumentos {tool_call.function.arguments}")
            print("=" * 20)

    elif step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        for content_block in message.content:
            if content_block.type == "text":
                print(f"Mensagem do Assistente:\n{content_block.text.value}")
            elif content_block.type == "image_file":
                file_id = content_block.image_file.file_id
                try:
                    image_data = client.files.content(file_id).read()
                    file_path = os.path.join("files", f"{file_id}.png")
                    with open(file_path, "wb") as f:
                        f.write(image_data)
                    print(f"Imagem {file_id} salva em: {file_path}")
                except Exception as e:
                    print(f"Erro ao salvar imagem {file_id}: {e}")


==== Passo: message_creation ====


C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_15932\1458713168.py:20: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Mensagem do Assistente:
Vamos primeiro ler o arquivo que você enviou para entender melhor os dados e identificar a coluna que representa a linha de produtos e as vendas associadas a cada linha. Em seguida, poderemos criar um gráfico de pizza com o percentual de vendas por linha de produtos.

==== Passo: tool_calls ====
Entrada do Interpretador de Código:
import pandas as pd

# Lendo o arquivo
file_path = '/mnt/data/file-VgWxcrqru8FsJSxD7zzDZf'
data = pd.read_csv(file_path)

# Exibindo as primeiras linhas do dataframe para entender sua estrutura
data.head()

==== Passo: message_creation ====
Mensagem do Assistente:
Os dados contêm várias colunas, incluindo "Product line" e "Total", que são as que precisamos para criar o gráfico de pizza. Vamos calcular o percentual de vendas por linha de produtos e, em seguida, gerar o gráfico de pizza.

==== Passo: tool_calls ====
Entrada do Interpretador de Código:
import matplotlib.pyplot as plt

# Calculando o total de vendas por linha de produtos
s